In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Configuration
batch_size=256
epochs=30
latent_dim=64
num_samples=2979
data_path = "data/hin.txt"

In [3]:
# Data Preprocessing

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 2979
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [4]:
# LSTM Model
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

lstm = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

lstm.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 70)]           0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 90)]           0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 256),                334848    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [6]:
# GRU Model
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.GRU(latent_dim, return_state=True)
encoder_outputs, state_h= encoder(encoder_inputs)

encoder_states = [state_h]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

gru = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

gru.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None, 70)]           0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None, 90)]           0         []                            
                                                                                                  
 gru (GRU)                   [(None, 256),                251904    ['input_3[0][0]']             
                              (None, 256)]                                                        
                                                                                                  
 gru_1 (GRU)                 [(None, None, 256),          267264    ['input_4[0][0]',       

In [7]:
lstm.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
lstm.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
history_lstm = lstm.save("lstm.keras")

Epoch 1/20
38/38 [==============================] - 4s 47ms/step - loss: 1.3074 - accuracy: 0.7854 - val_loss: 2.1405 - val_accuracy: 0.6848
Epoch 2/20
38/38 [==============================] - 1s 31ms/step - loss: 1.0258 - accuracy: 0.8074 - val_loss: 1.6027 - val_accuracy: 0.6873
Epoch 3/20
38/38 [==============================] - 1s 30ms/step - loss: 0.8718 - accuracy: 0.8090 - val_loss: 1.3770 - val_accuracy: 0.6873
Epoch 4/20
38/38 [==============================] - 1s 31ms/step - loss: 0.8425 - accuracy: 0.8090 - val_loss: 1.4413 - val_accuracy: 0.6864
Epoch 5/20
38/38 [==============================] - 1s 31ms/step - loss: 0.8196 - accuracy: 0.8091 - val_loss: 1.3776 - val_accuracy: 0.6864
Epoch 6/20
38/38 [==============================] - 1s 31ms/step - loss: 0.8072 - accuracy: 0.8088 - val_loss: 1.3353 - val_accuracy: 0.6873
Epoch 7/20
38/38 [==============================] - 1s 31ms/step - loss: 0.7990 - accuracy: 0.8085 - val_loss: 1.3008 - val_accuracy: 0.6872
Epoch 8/20
38

In [8]:
gru.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
gru.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
history_gru = gru.save("gru.keras")

Epoch 1/20
38/38 [==============================] - 3s 40ms/step - loss: 1.3760 - accuracy: 0.7865 - val_loss: 1.6819 - val_accuracy: 0.6869
Epoch 2/20
38/38 [==============================] - 1s 27ms/step - loss: 0.9611 - accuracy: 0.8070 - val_loss: 1.7303 - val_accuracy: 0.6848
Epoch 3/20
38/38 [==============================] - 1s 27ms/step - loss: 0.9048 - accuracy: 0.8078 - val_loss: 1.5078 - val_accuracy: 0.6869
Epoch 4/20
38/38 [==============================] - 1s 28ms/step - loss: 0.8420 - accuracy: 0.8086 - val_loss: 1.3440 - val_accuracy: 0.6872
Epoch 5/20
38/38 [==============================] - 1s 28ms/step - loss: 0.7973 - accuracy: 0.8070 - val_loss: 1.2990 - val_accuracy: 0.6866
Epoch 6/20
38/38 [==============================] - 1s 27ms/step - loss: 0.7591 - accuracy: 0.8118 - val_loss: 1.2633 - val_accuracy: 0.6915
Epoch 7/20
38/38 [==============================] - 1s 27ms/step - loss: 0.7227 - accuracy: 0.8201 - val_loss: 1.2003 - val_accuracy: 0.7007
Epoch 8/20
38

In [13]:
# Define sampling models for LSTM
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("lstm.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(([target_seq] + states_value), verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        # 
        if sampled_char == "\n"or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [16]:
encoder_model = keras.Model(encoder_inputs, encoder_states)
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [17]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Wow!
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Duck!
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Duck!
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Help!
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Jump.
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Jump.
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 15ms/step
-
Input sentence: Jump.
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hello!
Decoded sentence: मैं के का का का है।

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Hello!
Dec